In [2]:
# from google.colab import drive
# import os
# drive.mount("/content/drive")
# os.chdir("/content/drive/MyDrive/Libraries")

from sklearn.model_selection import train_test_split
import sys
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import Input, Model, Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator
from tensorflow.keras.layers import Layer, Dense, Flatten, Dropout, Concatenate, Add, Dot, Multiply, Reshape, Activation, BatchNormalization, SimpleRNNCell, RNN, SimpleRNN, LSTM, Embedding, Bidirectional, TimeDistributed, Conv1D, Conv2D, MaxPool1D, MaxPool2D, GlobalMaxPool1D, GlobalMaxPool2D, AveragePooling1D, AveragePooling2D, GlobalAveragePooling1D, GlobalAveragePooling2D, ZeroPadding2D
from tensorflow.keras.optimizers import SGD, Adam, Adagrad
from tensorflow.keras.metrics import MeanSquaredError, RootMeanSquaredError, MeanAbsoluteError, MeanAbsolutePercentageError, BinaryCrossentropy, CategoricalCrossentropy, SparseCategoricalCrossentropy, CosineSimilarity
from tensorflow.keras.layers.experimental.preprocessing import Rescaling
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.activations import linear, sigmoid, relu
from tensorflow.keras.initializers import RandomNormal, glorot_uniform, he_uniform, Constant
from tensorflow.keras.models import load_model

In [12]:
(X_tr_val, y_tr_val), (X_te, y_te) = tf.keras.datasets.cifar10.load_data()
X_tr, X_val, y_tr, y_val = train_test_split(X_tr_val, y_tr_val, test_size=0.2, shuffle=True)

# Modeling

In [39]:
# 18-layered ResNet
def residual_block(x, filters): 
    z1 = Conv2D(filters=filters, kernel_size=3, strides=1, padding="same")(x)
    z1 = BatchNormalization()(z1)
    z1 = Activation("relu")(z1)
    z1 = Conv2D(filters=filters, kernel_size=3, strides=1, padding="same")(z1)
    z1 = BatchNormalization()(z1)
    z1 = Activation("relu")(z1)

    z2 = Conv2D(filters=filters, kernel_size=1, strides=1, padding="same")(x)
    z2 = BatchNormalization()(z2)

    z = Activation("relu")(z1 + z2)
    return z

inputs = Input(shape=(32, 32, 3))

z = Conv2D(filters=64, kernel_size=7, strides=2, padding="valid")(inputs)
z = MaxPool2D(pool_size=3, strides=2, padding="same")(z)
for _ in range(2):
    z = residual_block(z, 64)
for _ in range(2):
    z = residual_block(z, 128)
for _ in range(2):
    z = residual_block(z, 256)
for _ in range(2):
    z = residual_block(z, 512)
z = GlobalAveragePooling2D()(z) 

outputs = Dense(units=10, activation="softmax")(z)

model = Model(inputs=inputs, outputs=outputs)
model.summary()

# # 50-layered ResNet
# def residual_block(x, filters): 
#     z1 = Conv2D(filters=filters[0], kernel_size=1, strides=1, padding="same")(x) 
#     z1 = BatchNormalization()(z1)
#     z1 = Activation("relu")(z1)
#     z1 = Conv2D(filters=filters[0], kernel_size=3, strides=1, padding="same")(z1)
#     z1 = BatchNormalization()(z1)
#     z1 = Activation("relu")(z1)
#     z1 = Conv2D(filters=filters[1], kernel_size=1, strides=1, padding="same")(z1)
#     z1 = BatchNormalization()(z1)
#     z1 = Activation("relu")(z1)

#     z2 = Conv2D(filters=filters[1], kernel_size=1, strides=1, padding="same")(x)
#     z2 = BatchNormalization()(z2)

#     z = Activation("relu")(z1 + z2)
#     return z

# inputs = Input(shape=(32, 32, 3))

# z = Conv2D(filters=64, kernel_size=7, strides=2, padding="valid")(inputs)
# z = MaxPool2D(pool_size=3, strides=2, padding="same")(z)
# for _ in range(3):
#     z = residual_block(z, filters=[64, 256])
# for _ in range(4):
#     z = residual_block(z, filters=[128, 512])
# for _ in range(6):
#     z = residual_block(z, filters=[256, 1024])
# for _ in range(3):
#     z = residual_block(z, filters=[512, 2048])
# z = GlobalAveragePooling2D()(z) 

# outputs = Dense(units=10, activation="softmax")(z)

# model = Model(inputs=inputs, outputs=outputs)
# model.summary()

Model: "model_9"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_12 (InputLayer)          [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 conv2d_229 (Conv2D)            (None, 13, 13, 64)   9472        ['input_12[0][0]']               
                                                                                                  
 max_pooling2d_11 (MaxPooling2D  (None, 7, 7, 64)    0           ['conv2d_229[0][0]']             
 )                                                                                                
                                                                                                  
 conv2d_230 (Conv2D)            (None, 7, 7, 64)     36928       ['max_pooling2d_11[0][0]'] 

In [ ]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["acc"])

es = EarlyStopping(monitor="val_loss", mode="auto", verbose=1, patience=2)
model_path = "resnet_18_cifar10.h5"
mc = ModelCheckpoint(filepath=model_path, monitor="val_acc", mode="auto", verbose=1, save_best_only=True)
gen = ImageDataGenerator(rescale=1/255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
hist = model.fit_generator(data=gen.flow(X_tr, y_tr, batch_size=32), validation_data=(x_val, y_val), epochs=10, callbacks=[es, mc])

Model: "model_13"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_16 (InputLayer)           [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv2d_448 (Conv2D)             (None, 13, 13, 64)   9472        input_16[0][0]                   
__________________________________________________________________________________________________
max_pooling2d_15 (MaxPooling2D) (None, 7, 7, 64)     0           conv2d_448[0][0]                 
__________________________________________________________________________________________________
conv2d_449 (Conv2D)             (None, 7, 7, 64)     36928       max_pooling2d_15[0][0]           
___________________________________________________________________________________________

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 8))
axes[0].plot(hist.history["loss"][1:], label="loss");
axes[0].plot(hist.history["val_loss"][1:], label="val_loss");
axes[0].legend();

axes[1].plot(hist.history["acc"][1:], label="acc");
axes[1].plot(hist.history["val_acc"][1:], label="val_acc");
axes[1].legend();

In [40]:
# import matplotlib.pyplot as plt

# acc = history.history["accuracy"]
# val_acc = history.history["val_accuracy"]

# loss = history.history["loss"]
# val_loss = history.history["val_loss"]

# epochs_range = range(10)

# plt.figure(figsize=(8, 8))
# plt.subplot(1, 2, 1)
# plt.plot(epochs_range, acc, label="Training Accuracy")
# plt.plot(epochs_range, val_acc, label="Validation Accuracy")
# plt.legend(loc="lower right")
# plt.title("Training and Validation Accuracy")

# plt.subplot(1, 2, 2)
# plt.plot(epochs_range, loss, label="Training Loss")
# plt.plot(epochs_range, val_loss, label="Validation Loss")
# plt.legend(loc="upper right")
# plt.title("Training and Validation Loss")
# plt.show()